## For the Total_EMI

In [3]:
import pandas as pd

# Input file path
file_path = "emi_clean.xlsx"

# Load the Excel file
xls = pd.ExcelFile(file_path)
df = pd.read_excel(xls)

# Rename columns 
df.columns = ["ID", "Value"]

# Convert to dictionary applying the logic to replace 0 values
dictionary = {}
previous_value = None
previous_id = None

for index, row in df.iterrows():
    current_id = row["ID"]
    current_value = row["Value"]

    if current_value == 0 and previous_value is not None and previous_id is not None:
        current_value = (previous_value / previous_id) * current_id

    dictionary[current_id] = current_value

    # Update previous values
    previous_value = current_value
    previous_id = current_id

# Replace values in the original table
for index, row in df.iterrows():
    current_id = row["ID"]
    if row["Value"] == 0 or row["Value"] > 10000:
        df.at[index, "Value"] = dictionary.get(current_id, row["Value"])

# Convert the dictionary into a DataFrame to export it to Excel
df_export = pd.DataFrame(dictionary.items(), columns=["ID", "Value"])

# Save
output_path = "total_emi_cleaned.xlsx"
df.to_excel(output_path, index=False)

## For the Amount Invested

In [4]:
import pandas as pd

def adjust_amount_invested_column(file):
    # Read the Excel file into a DataFrame
    df = pd.read_excel(file)
    
    # Ensure the columns are numeric
    df = df.apply(pd.to_numeric, errors='coerce')
    
    # Adjust values in column B if they exceed 100% of the monthly income, capping it at 20%
    df.iloc[:, 1] = df.iloc[:, 1].where(df.iloc[:, 1] <= df.iloc[:, 0], 0.2 * df.iloc[:, 0])
    
    # Save the changes to a new file
    df.to_excel('invested_data_clean.xlsx', index=False)
    

adjust_amount_invested_column('invested_data.xlsx')
